In [46]:
# ! [ -e /content ] && pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()
# #hide
# from fastai.vision.all import *
# from fastbook import *

# matplotlib.rc('image', cmap='Greys')

In [47]:
import torch
from torch import nn

In [48]:
from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage
from torch.utils.data import DataLoader, Subset

In [49]:
datasets.MNIST

torchvision.datasets.mnist.MNIST

In [50]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cuda device


In [51]:
class NossonNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(28*28, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.layers(x)
        return logits


In [52]:
model = NossonNet().to(device)

In [53]:
print(model)

NossonNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (layers): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [54]:
loss_fn = nn.CrossEntropyLoss()

In [55]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [56]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

In [57]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [58]:
bs = 64
train_dl = DataLoader(training_data, batch_size=(bs))
test_dl = DataLoader(test_data, batch_size=(bs))

In [59]:
for X, y in test_dl:
    print(f"x shape: {X.shape}")
    print(f"y shape: {y.shape} {y.dtype}")
    break

x shape: torch.Size([64, 1, 28, 28])
y shape: torch.Size([64]) torch.int64


In [60]:
training_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [61]:
# sample_idx = 7
# for batch, (X, y) in enumerate(train_dl):
#     print(f'batch number: {batch}')
#     print(f'X shape: {X.shape}')
#     print(f'y shape: {y.shape}')
#     print(f'sample:  {y[sample_idx]}')
#     ToPILImage()(X[sample_idx]).show()
#     break

In [62]:
def train(dataloader, model, loss_fn, optimizer): # trains a single epoch
    size = len(dataloader.dataset)
    model.train() # set model to train mode - needed for accumulating batch norms averages or training augmentations
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model.forward(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [63]:
def test(dataloader, model, loss_fn): # tests a single epoch
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    
    model.eval() # set to eval mode - use accumulated batch norm avg instead of aggregating, no augmentation (unless applying Test Time Augmentation)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

            

In [64]:
epochs = 5


## Train

In [65]:
for t in range(epochs):
    print(f"Epoch: {t+1} \n-------------")
    train(train_dl, model, loss_fn, optimizer)
    test(test_dl, model, loss_fn)

Epoch: 1 
-------------
loss: 2.301192  [   64/60000]
loss: 0.240496  [ 6464/60000]
loss: 0.181025  [12864/60000]
loss: 0.267344  [19264/60000]
loss: 0.120848  [25664/60000]
loss: 0.317544  [32064/60000]
loss: 0.130290  [38464/60000]
loss: 0.198728  [44864/60000]
loss: 0.251818  [51264/60000]
loss: 0.153502  [57664/60000]
Test Error: 
 Accuracy: 95.8%, Avg loss: 0.134917 

Epoch: 2 
-------------
loss: 0.093497  [   64/60000]
loss: 0.098135  [ 6464/60000]
loss: 0.114694  [12864/60000]
loss: 0.076513  [19264/60000]
loss: 0.061863  [25664/60000]
loss: 0.137674  [32064/60000]
loss: 0.055569  [38464/60000]
loss: 0.138316  [44864/60000]
loss: 0.122107  [51264/60000]
loss: 0.090622  [57664/60000]
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.082836 

Epoch: 3 
-------------
loss: 0.043571  [   64/60000]
loss: 0.026245  [ 6464/60000]
loss: 0.022867  [12864/60000]
loss: 0.081887  [19264/60000]
loss: 0.017868  [25664/60000]
loss: 0.079609  [32064/60000]
loss: 0.045017  [38464/60000]
loss: 0.069448

In [66]:
training_data[0][0].shape, training_data[0][0], training_data[0][1]

(torch.Size([1, 28, 28]),
 tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000

In [67]:
nn.Flatten()(training_data[0][0]).shape

torch.Size([1, 784])

### Test Overfitting:

In [68]:
train_subset = Subset(training_data, list(range(50)))
test_subset = Subset(test_data, list(range(50)))
subset_train_loader = DataLoader(train_subset, batch_size=(10))
subset_test_loader = DataLoader(test_subset, batch_size=(10))

coopppy = NossonNet().to(device)

In [69]:
opt = torch.optim.Adam(coopppy.parameters(), lr=1e-3)

In [70]:
for t in range(epochs*10):
    print(f"Epoch: {t+1} \n-------------")
    train(subset_train_loader, coopppy, loss_fn, opt)
    test(subset_test_loader, coopppy, loss_fn)

Epoch: 1 
-------------
loss: 2.317378  [   10/   50]
Test Error: 
 Accuracy: 40.0%, Avg loss: 2.075789 

Epoch: 2 
-------------
loss: 1.742868  [   10/   50]
Test Error: 
 Accuracy: 52.0%, Avg loss: 1.665648 

Epoch: 3 
-------------
loss: 0.996013  [   10/   50]
Test Error: 
 Accuracy: 68.0%, Avg loss: 1.218384 

Epoch: 4 
-------------
loss: 0.393345  [   10/   50]
Test Error: 
 Accuracy: 72.0%, Avg loss: 0.959704 

Epoch: 5 
-------------
loss: 0.126207  [   10/   50]
Test Error: 
 Accuracy: 70.0%, Avg loss: 0.857299 

Epoch: 6 
-------------
loss: 0.029984  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 0.899412 

Epoch: 7 
-------------
loss: 0.008620  [   10/   50]
Test Error: 
 Accuracy: 62.0%, Avg loss: 0.990100 

Epoch: 8 
-------------
loss: 0.003419  [   10/   50]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.038255 

Epoch: 9 
-------------
loss: 0.001552  [   10/   50]
Test Error: 
 Accuracy: 60.0%, Avg loss: 1.056251 

Epoch: 10 
-------------
loss: 0.000871  [   1